In [0]:
from keras.datasets import mnist
from matplotlib import pyplot
from keras.utils import to_categorical
from keras import Sequential
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dense
from keras.optimizers import SGD
from sklearn.model_selection import KFold
from keras.models import load_model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

In [0]:
def load_dataset():
  (x_train,y_train),(x_test,y_test) = mnist.load_data()
  #for i in range(9):
  #pyplot.subplot(330 + 1 + i)
  #pyplot.imshow(x_train[i], cmap=pyplot.get_cmap('gray'))
  x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2],1)
  x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2],1)
  y_train = to_categorical(y_train)
  y_test = to_categorical(y_test)
  print(x_train.shape,x_test.shape)
  print(y_train.shape,y_test.shape)
  return (x_train,y_train,x_test,y_test)

In [0]:
def normalize(x_train,x_test):
  x_train = x_train.astype('float32')
  x_test = x_test.astype('float32')
  x_train = x_train/255.0
  x_test = x_test/255.0
  return (x_train,x_test)

In [0]:
def model_definition():
  model = Sequential()
  model.add(Conv2D(32, (3,3), activation='relu',kernel_initializer='he_uniform',input_shape =(28,28,1)))
  model.add(MaxPooling2D(2,2))
  model.add(Flatten())
  model.add(Dense(100,activation='relu',kernel_initializer='he_uniform'))
  model.add(Dense(10,activation='softmax'))
  opt = SGD(learning_rate=0.01,momentum=0.9)
  model.compile(optimizer=opt, metrics=['accuracy'],loss = 'categorical_crossentropy')
  return model


In [0]:
def evaluate_model(x_train,y_train, folds=5):
  scores, histories = list(),list()
  kfold = KFold(folds, shuffle=True,random_state=1)
  for itrain,itest in kfold.split(x_train):
    model = model_definition()
    trainX, trainY, testX, testY = x_train[itrain], y_train[itrain], x_train[itest], y_train[itest]
    history = model.fit(trainX,trainY,batch_size=32,epochs=10,validation_data=(testX,testY),verbose=0)
    _,accuracy = model.evaluate(testX,testY,verbose = 0)
    print('> %.3f' % (accuracy * 100.0))
    scores.append(accuracy)
    histories.append(history)
  model.save('digit_recognition.h5')
  return (scores,histories)

In [0]:
from numpy import mean
from numpy import std
def summarize_diagnostics(histories):
	for i in range(len(histories)):
		pyplot.subplot(2, 1, 1)
		pyplot.title('Cross Entropy Loss')
		pyplot.plot(histories[i].history['loss'], color='blue', label='train')
		pyplot.plot(histories[i].history['val_loss'], color='orange', label='test')
		pyplot.subplot(2, 1, 2)
		pyplot.title('Classification Accuracy')
		pyplot.plot(histories[i].history['accuracy'], color='blue', label='train')
		pyplot.plot(histories[i].history['val_accuracy'], color='orange', label='test')
	pyplot.show()


def summarize_performance(scores):
	print('Accuracy: mean=%.3f std=%.3f, n=%d' % (mean(scores)*100, std(scores)*100, len(scores)))
	pyplot.boxplot(scores)
	pyplot.show()

In [0]:
def evaluate_train_data():
	# load dataset
	trainX, trainY, testX, testY = load_dataset()
	# prepare pixel data
	trainX, testX = normalize(trainX, testX)
	# evaluate model
	scores, histories = evaluate_model(trainX, trainY)
	# learning curves
	summarize_diagnostics(histories)
	# summarize estimated performance
	summarize_performance(scores)
 
# entry point, run the test harness
evaluate_train_data()

In [0]:
def evaluate_test_data():
  trainX, trainY, testX, testY = load_dataset()
  trainX, testX = normalize(trainX, testX)
  model = load_model('digit_recognition.h5')
  _, acc = model.evaluate(testX,testY,verbose=1)
  print('>%.3f' % (acc * 100.0))
evaluate_test_data()

In [0]:
#Prediction
def load_image(filename):
	img = load_img(filename, grayscale=True, target_size=(28, 28))
	img = img_to_array(img)
	img = img.reshape(1, 28, 28, 1)
	img = img.astype('float32')
	img = img / 255.0
	return img

def predict_image():
	img = load_image('sample_image.png')
	model = load_model('digit_recognition.h5')
	digit = model.predict_classes(img)
	print(digit[0])
 
predict_image()